# Other Cellular Automata: Langton's ant and Elementary CA (16 points)

In [ ]:
import numpy as np
import itertools
import pygame

from pygame.locals import (
    K_UP,
    K_DOWN,
    K_LEFT,
    K_RIGHT,
    K_ESCAPE,
    KEYDOWN,
    QUIT,
)

# Langton's ant

Langton's ant is a system with very simple rules, but complex behaviour. It can be described as a CA with a two-dimensional grid of binary cells. 

* Cells can be either *black* (**1**) or *white* (**0**). 
* The initial grid contains only white cells and an arbitrarily located 'ant'. 
* The 'ant' is a special cell that travels across the grid.
* The 'ant' can move a cell ahead in one of four directions. The direction is determined according to the following rules:
    * At a white cell (**0**) 'ant' turns <span style="color:green">90 degree clockwise</span>, flips the colour of the current cell and moves one cell ahead.
    * At a black cell (**1**) 'ant' turns <span style="color:red">90 degrees counterclokwise</span>, flips the colour of the current cell and moves one cell ahead.
![image](https://upload.wikimedia.org/wikipedia/commons/0/09/LangtonsAntAnimated.gif)
Sources and more: [1](https://mathworld.wolfram.com/LangtonsAnt.html), [2](https://en.wikipedia.org/wiki/Langton%27s_ant), [3](https://upload.wikimedia.org/wikipedia/commons/0/09/LangtonsAntAnimated.gif).

## Langton's ant in Python (8 points)

* (4 points) Fill the missing parts of the `move` & `update` functions.
* (2 points) Find and download a free ant image, load it in the code. Modify the `turnClockwise` and `turnCounterClokwise` functions, so that the ant image is rotated according to the CA's rules.
* (2 point) Move the ant automatically every few ms (add a timer).
Please note that the point (0,0) is located in the left upper corner of the board.

In [9]:
class Ant:
    def nextDir(self, direction, turn):
        directionTurnDict = {
            'c': 1,
            'cc': -1
        }
        direction = (direction + directionTurnDict[turn]) % 4
        return direction

    def __init__(self, x, y, direction):
        # x,y: inital ant position
        # Directions: 0 - north, 1 - east, 2 - south, 3 - west
        self.x = x
        self.y = y
        self.previous_x = x
        self.previous_y = y
        self.directionVectorsMap = {
            0: (0, -1),
            1: (1, 0),
            2: (0, 1),
            3: (-1, 0)
        }
        self.direction = direction
        # Load ant image
        self.antImage = pygame.image.load('char.png') # TODO Find this file on the web

        
    def turnClockwise(self):
        self.direction = self.nextDir(self.direction, 'c')
        return


        
    def turnCounterClockwise(self):
        self.direction = self.nextDir(self.direction, 'cc')
        return


    def move(self, bound_x, bound_y):
        self.previous_x = self.x
        self.previous_y = self.y
        self.x = (self.x + self.directionVectorsMap[self.direction][0]) % bound_x
        self.y = (self.y + self.directionVectorsMap[self.direction][1]) % bound_y
        return
        

    
    def getAntImage(self):
        return self.antImage
    

In [28]:
class Grid:
    def __init__(self, size, ant):
        self.size = size
        self.bound_x = size
        self.bound_y = size
        self.grid = np.zeros((size,size))
        self.ant = ant
        
    def update(self):
        if self.grid[self.ant.x][self.ant.y] == 0:
            self.ant.turnClockwise()
        else:
            self.ant.turnCounterClockwise()
        self.ant.move(self.bound_x, self.bound_y)
        self.grid[self.ant.previous_x][self.ant.previous_y] = (self.grid[self.ant.previous_x][self.ant.previous_y] + 1) % 2

        
    
    

In [29]:
def drawGrid(screen, grid, w_width, w_height):
    black = (0,0,0)
    white = (255,255,255)
    ant_colour = (128,128,128)
    
    size = grid.size
    blockSize = (min(w_width, w_height) - size) / size
    
    for x in range(0, size):
        for y in range(0, size):
            pos_x = (blockSize+1) * x
            pos_y = (blockSize+1) * y
            rect = pygame.Rect(pos_x, pos_y, blockSize, blockSize)
            # Draw cells
            if grid.grid[x][y] == 1:
                pygame.draw.rect(screen, black, rect, 0)
            else:
                pygame.draw.rect(screen, white, rect, 0)    
    
            
            if grid.ant.x == x and grid.ant.y == y:
                # Draw ant
                ant_x = (blockSize+1) * grid.ant.x
                ant_y = (blockSize+1) * grid.ant.y
                ant_img = grid.ant.getAntImage()
                ant_img = pygame.transform.scale(ant_img, (blockSize, blockSize))
                ant_img = pygame.transform.rotate(ant_img, grid.ant.direction * 90)
                screen.blit(ant_img, rect)
    pygame.display.flip()



In [ ]:
pygame.init()
# Parameters
w_width = 800
w_height = 800
gridSize = 40

# Set up the drawing window, adjust the size
screen = pygame.display.set_mode([w_width, w_height])

# Create ant & grid
ant = Ant(int(gridSize/2),int(gridSize/2),0)
grid = Grid(gridSize, ant)

# Set background
screen.fill((128, 128, 128))

drawGrid(screen, grid, w_width, w_height)
    
running = True

# TODO Add timer somewhere here and modify while loop shown below

while running:
    for event in pygame.event.get():   
        if event.type == QUIT:
            running = False
        
        if event.type == KEYDOWN:
            grid.update()
            drawGrid(screen, grid, w_width, w_height)



pygame.quit()

# Elementary Cellular Automaton (8 points)

Let's consider a very simple CA with a one-dimensional array of binary values. Let's assume that a cell has only two neighbours (i.e. the cell on the left and the cell on the right). In that case there are only 8 ($2^3$) possible  combinations of any three consecutive cells states. As a consequence the rules for this automaton are often presented in the following form (the first row describes currently processed cell and its neighbours, the second shows the new value):
![image](https://mathworld.wolfram.com/images/eps-gif/ElementaryCA30Rules_750.gif)

(source: [Wolfram Math World](https://mathworld.wolfram.com/ElementaryCellularAutomaton.html)).

E.g. The first rule says that if the cell and both its neighbours are black, then the new value will also be black.

In this system only $2^8$ possible sets of rules exist. The rule shown above is called *Rule 30* because the order of 1s and 0s resembles a binary number $00011110_2$, which in the decimal system is equal to 30.

The evolution of elementary cellular automata is often presented in a row-by-row manner:
![image](https://mathworld.wolfram.com/images/eps-gif/ElementaryCA30_1000.gif)

(source: [Wolfram Math World](https://mathworld.wolfram.com/ElementaryCellularAutomaton.html)).




## Elementary Cellular Automaton in Python
* (3 poinst) Implement the *Rule 30* elementary cellular automaton.
* (4 points) Create a rule-independent implementation (rule is passed as a string in `__init__`; 2 points). Use binary operators (2 points)
* (1 point)  Check  patterns for rules: 60, 62, 182, 250


In [38]:
class ElementaryCA:
    def __init__(self, length, rule):
        self.length = length
        self.rule = rule
        self.state = np.zeros((length), dtype = int)
        self.state[int(length/2)] = 1
        self.rowNumber = 0
        
    def nextRow(self):
            nextRow = np.zeros(self.length, dtype = int)
            #Boundary conditions; we're omitting the first and the last element
            for i in range(1,self.length-1):
                nextRow[i] = self.nextValue(self.state[i-1], self.state[i], self.state[i+1])
            self.state = nextRow
            self.rowNumber += 1
            
    def nextValue(self, prev, mid, nxt):
        string_val = str(prev) + str(mid) + str(nxt)
        val = int(string_val, 2)
        value_to_set = (self.rule >> val) & 1
        return value_to_set

In [36]:
def drawAndUpdate(screen, ca, w_width, w_height):
    black = (0,0,0)
    white = (255,255,255)    
    length = ca.length
    blockSize = (min(w_width, w_height) - length) / length
    
    for x in range(0, length):
        pos_x = (blockSize+1) * x
        pos_y = (blockSize+1) * ca.rowNumber
        rect = pygame.Rect(pos_x, pos_y, blockSize, blockSize)
        # Draw cells
        if ca.state[x] == 1:
            pygame.draw.rect(screen, black, rect, 0)
        else:
            pygame.draw.rect(screen, white, rect, 0)    

    pygame.display.flip()
    ca.nextRow()




In [ ]:
pygame.init()
# Parameters
w_width = 800
w_height = 800
# Set up the drawing window, adjust the size
screen = pygame.display.set_mode([w_width, w_height])

# Create ant & grid
ca = ElementaryCA(40, 250)

# Set background
screen.fill((128, 128, 128))

drawAndUpdate(screen, ca, w_width, w_height)
    
running = True

while running:
    for event in pygame.event.get():   
        if event.type == QUIT:
            running = False
        
        if event.type == KEYDOWN:
            if event.key == K_RIGHT:
                drawAndUpdate(screen, ca, w_width, w_height)
        
pygame.quit()